In [1]:
import os
import itertools
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.cluster import MiniBatchKMeans, Birch
from sklearn.model_selection import train_test_split


# The length of prioritized list 
PRIORITIZED_SIZE = 1000

# Clustering algorithms and parameters 
CLUSTERING_ALGOS = {
    'MiniBatchKMeans',
    'Birch',
}

N_CLUSTERS = 10
BATCH_SIZE = 1024
RANDDOM_STATE = 0
BIRCH_THRESHOLD = 0.5
BIRCH_BRACNCHING_FACTOR = 50

# Fingerprint feature specifications 
FP_RADIUS = [2, 3]
FP_N_BITS = 1024

# Local and FTP addresses 
PC_ROOT = './data/PubChem/'
PCBA_CID_FILE_NAME = 'Cid2BioactivityLink'
CID_INCHI_FILE_NAME = 'CID-InChI-Key'
PCBA_CID_SMILES_FILE_NAME = 'PCBA_CID-SMILES.csv'
PCBA_CID_FP_FILE_NAME = 'PCBA_CID-FP(%s).csv'

PCBA_CID_FTP_ADDRESS = \
    'ftp://ftp.ncbi.nlm.nih.gov/pubchem/Bioassay/Extras/%s.gz' % \
    PCBA_CID_FILE_NAME
CID_INCHI_FTP_ADDRESS = \
    'ftp://ftp.ncbi.nlm.nih.gov/pubchem/Compound/Extras/%s.gz' % \
    CID_INCHI_FILE_NAME

PCBA_CID_FILE_PATH = PC_ROOT + PCBA_CID_FILE_NAME
CID_INCHI_FILE_PATH = PC_ROOT + CID_INCHI_FILE_NAME
PCBA_CID_SMILES_FILE_PATH = PC_ROOT + PCBA_CID_SMILES_FILE_NAME


# PCBA_CID_SMILES_FILE_PATH = PC_ROOT + PCBA_CID_FP_FILE_NAME \
#                             % FINGERPRINT_N_BIT

try:
    os.makedirs(PC_ROOT)
except FileExistsError:
    pass

In [2]:
# Download and unpack the data 
if not os.path.exists(PCBA_CID_FILE_PATH):
    os.system('wget -r -nd -nc %s -P %s' 
              % (PCBA_CID_FTP_ADDRESS, PC_ROOT));

if not os.path.exists(CID_INCHI_FILE_PATH):
    os.system('wget -r -nd -nc %s -P %s' 
              % (CID_INCHI_FTP_ADDRESS, PC_ROOT));
    
os.system('find %s -type f -iname \"*.gz\" -exec gunzip {} +'
          % PC_ROOT);

In [3]:
# Read PCBA CID and SMILES strings 
pcba_cid_array = pd.read_csv(PCBA_CID_FILE_PATH, 
                             sep='\t',
                             header=0,
                             index_col=None,
                             usecols=[0, ])['CID'].unique()

if not os.path.exists(PCBA_CID_SMILES_FILE_PATH):

    pcba_cid_list = []
    pcba_smiles_list = []
    
    for cid_inchi_df_chunk in pd.read_csv(CID_INCHI_FILE_PATH,
                                          sep='\t',
                                          header=None,
                                          index_col=[0],
                                          usecols=[0, 1],
                                          chunksize=4096):
        
        cid_inchi_df_chunk = cid_inchi_df_chunk[
            cid_inchi_df_chunk.index.isin(pcba_cid_array)]
        
        for cid, row in cid_inchi_df_chunk.iterrows():
            
            inchi = row[1]
            try:
                mol = Chem.MolFromInchi(inchi)
                assert mol
            except AssertionError:
                continue
            
            pcba_cid_list.append(cid)
            pcba_smiles_list.append(Chem.MolToSmiles(mol))
    
    pcba_cid_smiles_df = pd.DataFrame(pcba_smiles_list, 
                                      index=pcba_cid_list)
    pcba_cid_smiles_df.columns = ['SMILES']
    pcba_cid_smiles_df.to_csv(PCBA_CID_SMILES_FILE_PATH)

else:
    pcba_cid_smiles_df = pd.read_csv(PCBA_CID_SMILES_FILE_PATH,
                                     index_col=[0])
print('%i/%i (%.2f%%) InChI converted to SMILES'
      % (len(pcba_cid_smiles_df), len(pcba_cid_array),
         100. * len(pcba_cid_smiles_df) / len(pcba_cid_array)))

3407702/3407889 (99.99%) InChI converted to SMILES


/home/xduan7/anaconda3/envs/pytorch/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
# Get features from the list of PCBA compounds
# Features includes ECFP and FCFP of certain radius 
for radius in FP_RADIUS:
    for useFeatures in [True, False]:
        
        spec = 'nBits=%i, radius=%i, useFeatures=%s' \
               % (FP_N_BITS, radius, useFeatures)
        pcba_cid_fp_file_name = PCBA_CID_FP_FILE_NAME % spec
        pcba_cid_fp_file_path = PC_ROOT + pcba_cid_fp_file_name
        
        if os.path.exists(pcba_cid_fp_file_path):
            continue
        
        pcba_cid_list = []
        pcba_fp_list = []
        for cid, row in pcba_cid_smiles_df.iterrows():
            mol = Chem.MolFromSmiles(row['SMILES'])

            if mol is None:
                pass
            else:
                binary = AllChem.GetMorganFingerprintAsBitVect(
                    mol, radius=radius, nBits=FP_N_BITS,
                    useFeatures=useFeatures).ToBitString()
                
                fp = hex(int(binary, 2))
                # h = hex(int(b, 2))
                # b = bin(int(h, 16))
                
                pcba_cid_list.append(cid)
                pcba_fp_list.append(fp)
        
        pcba_cid_fp_df = pd.DataFrame(pcba_fp_list,
                                      index=pcba_cid_list)
        pcba_cid_fp_df.columns = ['Fingerprint']
        pcba_cid_fp_df.to_csv(pcba_cid_fp_file_path)

In [5]:
feature_specs = \
    ['nBits=%i, radius=%i, useFeatures=%s' % combo for combo in 
     list(itertools.product([FP_N_BITS, ], FP_RADIUS, [False, True]))]

total_labels = []
for spec in feature_specs:
    
    # Prepare for the features that corresponds to the specs
    pcba_cid_fp_file_name = PCBA_CID_FP_FILE_NAME % spec
    pcba_cid_fp_file_path = PC_ROOT + pcba_cid_fp_file_name
    pcba_cid_fp_df = pd.read_csv(pcba_cid_fp_file_path, 
                                 index_col=[0], 
                                 dtype={'Fingerprint': str})
    
    pcba_cid_list = pcba_cid_fp_df.index.tolist()
    pcba_cid_fp_list = pcba_cid_fp_df['Fingerprint'].tolist()
    pcba_fp_features = []
    
    for i, hex_str in enumerate(pcba_cid_fp_list):
        binary_str = str(bin(int(hex_str, 16)))[2:]
        feature = np.concatenate(
            (np.zeros(shape=(1024 - len(binary_str))),
             np.array([np.int(c) for c in binary_str])), axis=0)
    
        pcba_fp_features.append(feature.astype(np.int8))
    pcba_fp_features = np.array(pcba_fp_features)
    
    # Iterate through different clustering algorithms
    for cluster_algo in CLUSTERING_ALGOS:
        
        if cluster_algo == 'Birch':
            model = Birch(n_clusters=N_CLUSTERS, 
                          threshold=BIRCH_THRESHOLD,
                          branching_factor=BIRCH_BRACNCHING_FACTOR)
        elif cluster_algo == 'MiniBatchKMeans':
            model = MiniBatchKMeans(n_clusters=N_CLUSTERS,
                                    batch_size=BATCH_SIZE,
                                    random_state=RANDDOM_STATE)
        else:
            raise ValueError('No clustering algorithm defined as %s'
                             % cluster_algo)
        
        # Perform clustering algorithm by batch
        print('Clustering using %s on fingerprint features '
              '(spec=%s) ... ' % (cluster_algo, spec))
        
        for i in range(0, len(pcba_fp_features), BATCH_SIZE):
            end_index = np.min((i + BATCH_SIZE, len(pcba_fp_features)))
            batch_pcba_fp_features = pcba_fp_features[i: end_index]
            model.partial_fit(batch_pcba_fp_features)
        
        labels = np.array([])
        for i in range(0, len(pcba_fp_features), BATCH_SIZE):
            end_index = np.min((i + BATCH_SIZE, len(pcba_fp_features)))
            batch_pcba_fp_features = pcba_fp_features[i: end_index]

            labels = np.concatenate(
                (labels, 
                 model.predict(batch_pcba_fp_features)), axis=0)
        
        labels = labels.reshape((-1)).astype(np.int8)
        total_labels.append(labels)

total_labels = np.array(total_labels).T
np.savetxt(PC_ROOT + 'labels.csv', total_labels, delimiter="\t")

In [ ]:
# Divide the PCBA chemicals using 
# the clustering labels obtained from different clustering runs

# With train/test split for this task 
_, prioritized_pcba_cid_list, = train_test_split(
    pcba_cid_list, test_size=PRIORITIZED_SIZE, random_state=RANDDOM_STATE)


prioritized_pcba_cid_smiles_df = pcba_cid_smiles_df[
    pcba_cid_smiles_df.index.isin(prioritized_pcba_cid_list)]
pcba_cid_smiles_df.to_csv(PC_ROOT + 'Prioritized_PCBA_CID-SMILES.csv')